In [105]:
import numpy as np
import pandas as pd
import itertools
import random
import datetime as d
import hashlib as h
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from scipy.optimize import curve_fit
from sklearn.preprocessing import minmax_scale

# Global Parameters

In [106]:
# global parameters
global_misfire = 0.00        # malfunction rate
global_tagged = 0.00         # percent tagged malicious
global_tagged_rate = 0.00    # rate tagged nodes act maliciously
global_epsilon = 2           # base
global_k = 4                 # number of historical windows
global_history_len = np.sum([int(global_epsilon ** i) for i in range(1, global_k+1)])
global_clip_rate = 0.0       # deactivation threshold
global_sbe = 0.30             # slow boltzmann estimator cutoff

# test parameters
network_size = 1000
n_iter = 1000

In [107]:
# rng
rng = np.random.default_rng()

In [108]:
# scale data to bounds
def scaler(data = [], upper = 1, lower = 0.01, invert = False):
    if invert:
        inverted = minmax_scale(data, (lower, upper))
        return [(1-x) for x in inverted]
    return minmax_scale(data, (lower, upper))

# Utility Functions

In [109]:
# splits a historical record of accuracies into exponentially larger buckets
def exp_split(arr = [], epsilon = global_epsilon, n = global_k):
    # history length
    h_len = np.sum([int(epsilon ** i) for i in range(1, n+1)])
    # trim history
    recent = list(reversed(arr[-h_len:]))
    split_list = []
    lag_idx = 0
    for x in [int(epsilon ** i) for i in range(1, n+1)]:
        split_list.append(recent[lag_idx : lag_idx + x])
        lag_idx += x
    return [np.mean(x) for x in split_list]

# whm
def whm(arr = [], epsilon = global_epsilon, n = global_k):
    # w_i
    w_i = []
    for i in range(n, 0, -1):
        w_i.append(epsilon ** i)
    # sigma(w_i)
    sigma_w_i = np.sum(w_i)
    # x_i
    x_i = exp_split(arr, epsilon, n)
    # w_i / x_ i
    w_over_x = [a/b if b else 0 for (a, b) in zip(w_i, x_i)]
    # sigma(w_i)/(x_ i)
    sigma_w_over_x = np.sum(w_over_x)

    return sigma_w_i / sigma_w_over_x

# Ledger Structures

In [110]:
class Ledger:
    def __init__(self):
        self.blockchain = []
        self.n_blocks = 0
        self.blockchain.append(self.genesis())

    def genesis(self):
        return Ledger.Block(id = self.gen_id(),
                            index = 0,
                            time = d.datetime.now(),
                            data = 'Genesis Block',
                            lasthash = '')
        
    def snoop(self,
              id):
        history = []
        for b in self.blockchain:
            if str(id) in b.data:
                _, _, substr = b.data.partition(str(id) + ':')
                history.append(substr[:3])
        return history

    def get_reputation(self,
                       id):
        if len(self.snoop(id)) > global_history_len:
            return whm([float(i) for i in self.snoop(id)])
        else:
            print('Node history too short to calculate WHM with minimum history length of %f' % (global_history_len))


    # produce a unique ID for a new block
    def gen_id(self):
        _temp = rng.integers(low = 40000, high = 80000)
        while _temp in [x.id for x in self.blockchain]:
            _temp = rng.integers(low = 40000, high = 80000)
        return _temp

    def add_block(self, 
                  data = ''):
        _index = self.blockchain[-1].index + 1
        _time = d.datetime.now()
        _data = data
        _lasthash = self.blockchain[-1].hash
        self.n_blocks += 1
        self.blockchain.append(Ledger.Block(id = self.gen_id(),
                                            index = _index,
                                            time = _time,
                                            data = _data,
                                            lasthash = _lasthash))

    def print_ledger(self):
        for b in self.blockchain:
            print('Block ID: %d' % (b.id))
            print('\tIndex: %d' % (b.index))
            print('\tTime: %s' % (str(b.time)))
            print('\tData: %s' % (str(b.data)))
            print('\tLast Block: %s' % (str(b.lasthash)))
            print('\tHash: %s' % (str(b.hash)))
            print()


    class Block:
        def __init__(self,
                     id,
                     index, 
                     time, 
                     data, 
                     lasthash):
            self.id = id
            self.index = index
            self.time = time
            self.data = data
            self.lasthash = lasthash
            self.hash = self.hash_block()


        def encode(self):
            output = str(self.index) + str(self.time) + str(self.data) + str(self.lasthash)
            return output.encode('utf-8')

        def hash_block(self):
            encryption = h.sha256()
            encryption.update(self.encode())
            return encryption.hexdigest()

# Network Structures

In [111]:
class Node:
    def __init__(self,
                 ledger = '',
                 id = id,
                 starting_rep = 1.0,
                 misfire_rate = global_misfire,
                 tagged_rate = global_tagged_rate,
                 epsilon = global_epsilon,
                 n_windows = global_k):
        self.id = id
        self.local_copy = ledger

        # dictionary of connected nodes and corresponding weights
        self.active = 1
        self.connected = {}
        self.epsilon = epsilon
        self.n_windows = n_windows
        self.history_length = np.sum([int(self.epsilon ** i) for i in range(0, self.n_windows)])
        self.history = [starting_rep for x in range(self.history_length)]
        self.reputation = starting_rep
        self.misfire_rate = misfire_rate          # global misfire rate
        self.tagged = 1                           # good node = 1, bad node = 0
        self.tagged_rate = tagged_rate            # frequency tagged nodes act maliciously
        self.combined_rate = misfire_rate + tagged_rate

    # splits a historical record of accuracies into exponentially larger buckets
    def exp_split(self, arr = [], epsilon = global_epsilon, n = global_k):
        # history length
        h_len = np.sum([int(epsilon ** i) for i in range(1, n+1)])
        # trim history
        recent = list(reversed(arr[-h_len:]))
        split_list = []
        lag_idx = 0
        for x in [int(epsilon ** i) for i in range(1, n+1)]:
            split_list.append(recent[lag_idx : lag_idx + x])
            lag_idx += x
        return [np.mean(x) for x in split_list]

    # whm
    def whm(self, arr = [], epsilon = global_epsilon, n = global_k):
        # w_i
        w_i = []
        for i in range(n, 0, -1):
            w_i.append(epsilon ** i)
        # sigma(w_i)
        sigma_w_i = np.sum(w_i)
        # x_i
        x_i = self.exp_split(arr, epsilon, n)
        # w_i / x_ i
        w_over_x = [a/b if b else 0 for (a, b) in zip(w_i, x_i)]
        # sigma(w_i)/(x_ i)
        sigma_w_over_x = np.sum(w_over_x)

        return sigma_w_i / sigma_w_over_x

    # return weight of two nodes given reputations
    def get_weight(self,
                   v_i,
                   h_j):
        _mean = np.mean((v_i, h_j))
        return np.log2(_mean)

    def update_reputation(self):
        self.reputation = self.whm(self.history)

    def update_history(self,
                       update):
        self.history.append(update)

    def add_connection(self, 
                       other):
        w_ij = self.get_weight(self.reputation, other.reputation)
        self.connected.update({other:w_ij})

    def poll(self):
        # response based on global misfire rate
        if self.tagged:
            return 1.0 if rng.random() > self.misfire_rate else 0.5
        # response based on tagged rate + global misfire rate
        else:
            return 1.0 if rng.random() > self.combined_rate else 0.5

In [112]:
class Network:
    def __init__(self,
                 n_nodes,
                 starting_rep = 1.0,
                 mode = 0,
                 misfire_rate = global_misfire,
                 tagged = global_tagged,
                 tagged_rate = global_tagged_rate,
                 sbe_rate = global_sbe,
                 clip_rate = global_clip_rate):
        
        # ledger
        self.ledger = Ledger()
        # local reference to blockchain
        self.blockchain = self.ledger.blockchain

        # network attributes
        self.network_misfire = 0.0 if mode == 1 else misfire_rate
        self.mode = mode
        self.sbe = sbe_rate
        self.starting_rep = starting_rep

        # node lists
        self.n_nodes = n_nodes
        self.active_nodes = [Node(id = 1000 + i, 
                                  ledger = self.ledger,
                                  misfire_rate = self.network_misfire, 
                                  starting_rep = self.starting_rep) for i in range(self.n_nodes)]
        self.inactive_nodes = []

        # node behavior attributes
        self.clip_rate = clip_rate
        self.clipping = 1 if self.clip_rate != 0.0 else 0
        self.tagged_ratio = tagged
        self.tagged_rate = global_tagged_rate

        # network performance metrics
        self.n_active = n_nodes
        self.n_inactive = 0
        self.network_reputation = 1.0
        self.accepted = 0          # quorums accepted
        self.rejected = 0          # quorums rejected
        self.good_quorums = 0      # quorums that reached consensus
        self.bad_quorums = 0       # quorums that did NOT reach consensus
        self.pos_sbe_quorums = 0   # quorums with sbe estimates > threshold
        self.neg_sbe_quorums = 0   # quorums with sbe estimates < threshold
        self.good_miners = 0       # miners who polled 1
        self.bad_miners = 0        # miners who polled 0

        # network performance histories
        self.active_history = [n_nodes]
        self.inactive_history = [0]
        self.reputation_history = []
        self.accepted_history = [0]
        self.rejected_history = [0]
        self.good_quorum_history = [0]
        self.bad_quorum_history = [0]
        self.pos_sbe_history = [0]
        self.neg_sbe_history = [0]
        self.good_miner_history = [0]
        self.bad_miner_history = [0]

        self.update_network_state()


    def good_or_bad(self):
        return 1 if rng.random() > self.tagged_ratio else 0

    def tag_nodes(self):
        for nde in self.active_nodes:
            nde.tagged = self.good_or_bad()
        print('Network tagging successful... %.2f%% of nodes currently tagged...' % (self.tagged_ratio * 100))

    def update_mean_reputation(self):
        self.network_reputation = np.mean([nde.reputation for nde in self.active_nodes])
        self.reputation_history.append(self.network_reputation)
        print('Network mean reputation calculated: [ %.2f ]' % (self.network_reputation))

    def clip_nodes(self):
        for nde in self.active_nodes:
            if nde.reputation < self.clip_rate:
                nde.active = 0
                self.inactive_nodes.append(self.active_nodes.pop(self.active_nodes.index(nde)))
                self.n_active -= 1
                self.n_inactive += 1
        self.active_history.append(self.n_active)
        self.inactive_history.append(self.n_inactive)
        print('Network purge successful... %d nodes purged so far...' % (self.n_inactive))

    def update_network_reputations(self):
        for nde in self.active_nodes:
            nde.update_reputation()
        print('Network reputation updates successful...')

    def update_network_state(self):
        # update node reputations
        self.update_network_reputations()
        # update mean reputation
        self.update_mean_reputation()
        # clip nodes
        if self.clipping:
            self.clip_nodes()
    
    def select_quorum(self, 
                      count = 7):
        indexes = rng.integers(low = 0, high = self.n_active, size = count)
        selection = sorted([self.active_nodes[i] for i in indexes], key = lambda x: x.reputation, reverse = True)
        miner = selection.pop(6)
        print('Quorum selection successful...')
        return selection[:3], selection[3:], miner

    def update_histories(self):
        self.good_quorum_history.append(self.good_quorums)
        self.bad_quorum_history.append(self.bad_quorums)
        self.good_miner_history.append(self.good_miners)
        self.bad_miner_history.append(self.bad_miners)
        self.pos_sbe_history.append(self.pos_sbe_quorums)
        self.neg_sbe_history.append(self.neg_sbe_quorums)
        self.accepted_history.append(self.accepted)
        self.rejected_history.append(self.rejected)
        print('Network histories update successful...')

    def aggregate_data(self,
                       miner,
                       quorum,
                       q,
                       m):
        # add quorum update to data
        data = ''
        for n in quorum.visible_nodes + quorum.hidden_nodes:
            data += str(n.id) + ':' + str(q) + ', '
        # add miner update to data
        data += str(miner.id) + ':' + str(m)
        return data
        
    def run_quorum(self,
                   n_v = 3,
                   n_h = 3):
        # select quorum, miner
        num_nodes = n_v + n_h + 1
        v_nodes, h_nodes, miner = self.select_quorum(count = num_nodes)
        quorum = self.Quorum(v_nodes = v_nodes, h_nodes = h_nodes)
        # run quorum calculations
        quorum.calculate()
        # poll quorum
        q_poll, est = quorum.poll_quorum()
        # poll miner
        m_poll = miner.poll()
        # update miner
        miner.update_history(m_poll)
        # update quorum
        for nde in v_nodes + h_nodes:
            nde.update_history(q_poll)
        # update quorum counts
        if q_poll == 1.0:
            self.good_quorums += 1
        else:
            self.bad_quorums += 1
        # update miner counts
        if m_poll == 1.0:
            self.good_miners += 1
        else:
            self.bad_miners += 1
        # update sbe results
        if est > self.sbe:
            self.pos_sbe_quorums += 1
        else:
            self.neg_sbe_quorums += 1
        # update accepted/rejected
        if m_poll == 1.0 and q_poll == 1.0 and est > self.sbe:
            self.accepted += 1
        else:
            self.rejected += 1

        self.ledger.add_block(data = self.aggregate_data(miner = miner,
                                                         quorum = quorum,
                                                         q = q_poll,
                                                         m = m_poll))

    def run_iteration(self,
                      n_v = 3,
                      n_h = 3):
        # run quorum
        self.run_quorum(n_v = n_v, n_h = n_h)
        # update network state
        self.update_network_state()
        # update network histories
        self.update_histories()
        print('Iteration concluded successfully...\n')

    def print_metrics(self):
        print('Nodes:')
        print('\tActive: %d' % (self.n_active))
        print('\tInactive: %d' % (self.n_inactive))
        print('\tMean reputation: %.2f' % (self.network_reputation))
        print('Quorums:')
        print('\tAccepted: %d' % (self.accepted))
        print('\tRejected: %d' % (self.rejected))
        print('\tConsensus: %d' % (self.good_quorums))
        print('\tNo Consensus: %d' % (self.bad_quorums))
        print('\tAbove SBE: %d' % (self.pos_sbe_quorums))
        print('\tBelow SBE: %d' % (self.neg_sbe_quorums))
        print('\tGood Miners: %d' % (self.good_miners))
        print('\tBad Miners: %d' % (self.bad_miners))

    def export_metrics(self):
        records = {
            'active_history':self.active_history,
            'inactive_history':self.inactive_history,
            'reputation_history':self.reputation_history,
            'accepted_history':self.accepted_history,
            'rejected_history':self.rejected_history,
            'good_quorum_history':self.good_quorum_history,
            'bad_quorum_history':self.bad_quorum_history,
            'pos_sbe_history':self.pos_sbe_history,
            'neg_sbe_history':self.neg_sbe_history,
            'good_miner_history':self.good_miner_history,
            'bad_miner_history':self.bad_miner_history
        }
        return records

    def test(self, 
             n_iter,
             n_v = 3,
             n_h = 3):
        for i in range(n_iter):
            if self.n_active < 10:
                print('Terminating test...\n')
                break
            self.run_iteration(n_v = n_v, n_h = n_h)
        self.print_metrics()


    class Quorum:
        def __init__(self,
                     v_nodes,
                     h_nodes,
                     n_v = 3,
                     n_h = 3):
            self.num_visible = n_v
            self.num_hidden = n_h
            self.visible_nodes = v_nodes
            self.hidden_nodes = h_nodes
            # generate configurations
            self.alpha_config = [list(i) for i in itertools.product([1, 0], repeat = self.num_visible)]
            self.beta_config = [list(i) for i in itertools.product([1, 0], repeat = self.num_hidden)]
            # dictionaries for configuration calculation
            self.state_bias = {}
            self.state_weight = {}
            self.energy = {}
            self.e_energy = {}
            self.total_e_energy = 0
            self.joint_conditionals = {}
            self.joint_probabilities = {}
            # run initialization methods
            self.update_weights()

        # Configurations are stored in state_bias and state_weight as nested dicts
        #     - Keys reference to index in alpha_config and beta_config
        #     - Values are calculated state_bias or state_weight
        # Example:
        #     - visible node configuration [1, 1, 1] is index 0 in alpha_config
        #     - all beta configurations and corresponding values are in level 2 dict
        # 
        #     state_bias[0] = {0:val0, 1:val1, 2:val2, 3:val3, 4:val4, 5:val5, 6:val6, 7:val7}
        # 

        def update_weights(self):
            # update weights
            for vnode in self.visible_nodes:
                # link to all hidden nodes
                for hnode in self.hidden_nodes:
                    vnode.add_connection(hnode)
                    hnode.add_connection(vnode)
            print('Quorum weight update successful...')

        def calc_config_state_bias(self, v_idx, h_idx):
            v_sum = 0
            h_sum = 0
            # visible nodes
            for _a, i in zip(self.alpha_config[v_idx], range(len(self.visible_nodes))):
                v_sum += (_a * self.visible_nodes[i].reputation)
                continue
            # hidden nodes
            for _b, j in zip(self.beta_config[h_idx], range(len(self.hidden_nodes))):
                h_sum += (_b * self.hidden_nodes[j].reputation)
            return -(v_sum + h_sum)

        def calc_config_state_weight(self, a_config, b_config):
            # iterate through a_configs 1/0
            #     for each visible node
            #         multiply b_config by weight stored in node.connected
            sigma = 0
            # iterate through visible nodes
            for a, _n in zip(a_config, self.visible_nodes):
                # iterate through hidden nodes connected to visible nodes
                for b, _h in zip(b_config, list(_n.connected.values())):
                    sigma += (a * b * _h)
            return sigma

        def update_state_bias(self):
            for a_idx in range(len(self.alpha_config)):
                b_configs = {}
                # calculate all beta configs for specific alpha config
                for b_idx in range(len(self.beta_config)):
                    b_configs[b_idx] = self.calc_config_state_bias(a_idx, b_idx)  
                # add aggregated configs to state_bias dict
                self.state_bias[a_idx] = b_configs

        def update_state_weight(self):
            for a_idx in range(len(self.alpha_config)):
                b_configs = {}
                # calculate all beta configs for specific alpha config
                for b_idx in range(len(self.beta_config)):
                    b_configs[b_idx] = self.calc_config_state_weight(self.alpha_config[a_idx], self.beta_config[b_idx])
                # add aggregated configs to state_weight dict
                self.state_weight[a_idx] = b_configs

        def update_energy(self):
            for _a in range(len(self.alpha_config)):
                b_configs = {}
                for _b in range(len(self.beta_config)):
                    b_configs[_b] = -(self.state_bias[_a][_b] - self.state_weight[_a][_b])
                self.energy[_a] = b_configs
            
        def update_e_energy(self):
            for k, v in self.energy.items():
                b_configs = {}
                for _k, _v in v.items():
                    b_configs[_k] = np.exp(_v)
                self.e_energy[k] = b_configs
            # update total e_energy
            self.total_e_energy = np.sum([list(self.e_energy[x].values()) for x in self.e_energy.keys()])

        def update_joint_conditionals(self):
            for k, v in self.e_energy.items():
                b_configs = {}
                for _k, _v in v.items():
                    b_configs[_k] = _v / self.total_e_energy
                self.joint_conditionals[k] = b_configs

        def update_joint_probabilities(self):
            for k, v in self.joint_conditionals.items():
                self.joint_probabilities[k] = np.sum(list(v.values()))

        def get_estimate(self):
            return list(self.joint_probabilities.values())[0]

        def calculate(self):
            self.update_state_bias()
            self.update_state_weight()
            self.update_energy()
            self.update_e_energy()
            self.update_joint_conditionals()
            self.update_joint_probabilities()

        def query_nodes(self):
            return 0.5 if 0.5 in [nde.poll() for nde in self.visible_nodes] + [nde.poll() for nde in self.hidden_nodes] else 1.0

        def poll_quorum(self):
            return self.query_nodes(), self.get_estimate()

# Run Network

In [113]:
test_net = Network(n_nodes = 100, 
                   starting_rep = 1.0,
                   mode = 2,
                   misfire_rate = 0.05,
                   tagged = 0.1,
                   tagged_rate = 0.1,
                   sbe_rate = 0.3,
                   clip_rate = 0.6)

Network reputation updates successful...
Network mean reputation calculated: [ 1.00 ]
Network purge successful... 0 nodes purged so far...


In [114]:
test_net.test(1000)

Streaming output truncated to the last 5000 lines.
Network histories update successful...
Iteration concluded successfully...

Quorum selection successful...
Quorum weight update successful...
Network reputation updates successful...
Network mean reputation calculated: [ 0.89 ]
Network purge successful... 0 nodes purged so far...
Network histories update successful...
Iteration concluded successfully...

Quorum selection successful...
Quorum weight update successful...
Network reputation updates successful...
Network mean reputation calculated: [ 0.89 ]
Network purge successful... 0 nodes purged so far...
Network histories update successful...
Iteration concluded successfully...

Quorum selection successful...
Quorum weight update successful...
Network reputation updates successful...
Network mean reputation calculated: [ 0.89 ]
Network purge successful... 0 nodes purged so far...
Network histories update successful...
Iteration concluded successfully...

Quorum selection successful...

In [115]:
test_net.ledger.print_ledger()

Streaming output truncated to the last 5000 lines.
	Hash: f2feb0990dc64411f8d15be1067bc91a84cc6aac79b65ed0f8d1609568789764

Block ID: 52713
	Index: 287
	Time: 2022-11-29 17:43:18.239071
	Data: 1000:1.0, 1090:1.0, 1088:1.0, 1072:1.0, 1045:1.0, 1059:1.0, 1059:1.0
	Last Block: f2feb0990dc64411f8d15be1067bc91a84cc6aac79b65ed0f8d1609568789764
	Hash: 80fa1324e9f3ead6771930b7227d6b70de0d99eb6391298988152894f270a7bc

Block ID: 57118
	Index: 288
	Time: 2022-11-29 17:43:18.260344
	Data: 1009:1.0, 1070:1.0, 1010:1.0, 1032:1.0, 1060:1.0, 1019:1.0, 1053:1.0
	Last Block: 80fa1324e9f3ead6771930b7227d6b70de0d99eb6391298988152894f270a7bc
	Hash: a0ac871705a9ab984f1a8b010dfa5fe7c451960c842fde94916a15adb50eee93

Block ID: 79110
	Index: 289
	Time: 2022-11-29 17:43:18.276543
	Data: 1027:1.0, 1097:1.0, 1088:1.0, 1018:1.0, 1020:1.0, 1089:1.0, 1069:1.0
	Last Block: a0ac871705a9ab984f1a8b010dfa5fe7c451960c842fde94916a15adb50eee93
	Hash: 66390afb2e4c6d8d62279be553a560883103573c0c36a26d388c6391cbc347de

Block ID:

In [116]:
test_net.ledger.snoop(1031)

['1.0',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '0.5',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '0.5',
 '0.5',
 '1.0',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.5',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0']

In [117]:
test_net.ledger.get_reputation(1031)

0.981920199501247